In [ ]:
import sys 
sys.path.append("./src")

In [2]:
from argparse import ArgumentParser
from pathlib import Path
import os
import torch
import logging
import json
import random
import numpy as np
from collections import namedtuple
from tempfile import TemporaryDirectory

from torch.utils.data import DataLoader, Dataset, RandomSampler
from torch.utils.data.distributed import DistributedSampler
from tqdm import tqdm

from pytorch_transformers import WEIGHTS_NAME, CONFIG_NAME
from pytorch_transformers.modeling_bert import BertForPreTraining, BertForSequenceClassification
from pytorch_transformers.tokenization_bert import BertTokenizer
from pytorch_transformers.optimization import AdamW, WarmupLinearSchedule
from pathlib import Path
import re

import time
import pandas as pd
from sklearn.metrics import classification_report
import  matplotlib.pyplot as plt
import scipy

from pytorch_transformers import BertPreTrainedModel, BertModel, BertForSequenceClassification, BertConfig
import json
import math
import os
import sys
from io import open

import torch
from torch import nn
from torch.nn import CrossEntropyLoss, MSELoss
import gzip
logger = logging.getLogger(__name__)
import  matplotlib.pyplot as plt

from collections import defaultdict
import sklearn
from torch import functional as F

In [8]:
from tqdm import tqdm
import glob
from multiprocessing import Pool
import time
from sklearn.linear_model import LogisticRegression
import pickle
from multiprocessing import Pool, cpu_count
import requests
import copy
import ml_metrics 

In [247]:
read_file_name = "models/20200120_BERTMeanVEC/OnlyStock/pytorch_model.bin"

# use all labels
read_file_name = "models/20200120_BERTMeanVEC/Sector_1/01/pytorch_model.bin"
read_file_name = "models/20200120_BERTMeanVEC/Stock_Sector_1/01/pytorch_model.bin"
read_file_name = "models/20200120_BERTMeanVEC/Sector_SectorName_1/01_2/pytorch_model.bin"
read_file_name = "models/20200120_BERTMeanVEC/Stock_Sector_SectorName_1/01/pytorch_model.bin"

# use two  labels
read_file_name = "models/20200120_BERTMeanVEC/Sector_Name_2labels/01/pytorch_model.bin"
read_file_name = "models/20200120_BERTMeanVEC/Stock_Sector_Name_2labels/01/pytorch_model.bin"
# use  five labels
read_file_name = "models/20200120_BERTMeanVEC/Sector_Name_5labels/01/pytorch_model.bin"
read_file_name = "models/20200120_BERTMeanVEC/Stock_Sector_Name_5labels/01/pytorch_model.bin"

In [9]:
max_seq_length = 512

In [10]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [11]:
return_val_dict = pickle.load(open("data/return_val_dict.pkl", "rb"))
df_ticker = pd.read_csv("data/ticker_list.csv")
text_data_id_df = pd.read_csv("data/text_data_id.csv")
month_stock_info_df = pd.read_csv("data/stock_price.csv")

In [13]:
ticker2sector = dict(zip(df_ticker["ticker"], df_ticker["sector"]))
ticker2industry = dict(zip(df_ticker["ticker"], df_ticker["industry"]))
all_month_list = np.sort(list(set(month_stock_info_df["month"])))
month2index = dict(zip(all_month_list, range(len(all_month_list))))
text_data = dict(zip(text_data_id_df["ticker"], [
    [np.array(text_data_id_df["text_id"].iloc[i][1:-1].split(",")).astype(np.int32)] for i in range(len(text_data_id_df))]))

In [14]:
use_stock_price_list = []
sector_array = []
industry_array = []
use_text_data = []
use_ticker_list_data = []
stock_size = 59
for index, ticker in enumerate(df_ticker["ticker"]):
    if ticker in return_val_dict:
        if len(return_val_dict[ticker]) >= 12:
            if  (type(ticker2sector[ticker]) == str) :
                use_stock_price_list.append(return_val_dict[ticker][-stock_size:])
                sector_array.append(ticker2sector[ticker])
                use_text_data.append(text_data[ticker])
                industry_array.append(ticker2industry[ticker])
                use_ticker_list_data.append(ticker)

In [15]:
similarity_mat_df = pd.read_csv("../AnalysisEnglishAnnualReports/BertIndustryClustering/stock_similarity_mat/all_similarity_mat.csv", 
                                index_col=0)
similarity_mat_df = similarity_mat_df[df_ticker["ticker"]].T[df_ticker["ticker"]]

In [17]:
similarity_inner_mat = np.array(similarity_mat_df[
    [len(return_val_dict[ticker]) > 12 for index, ticker in enumerate(df_ticker["ticker"])]].T[
    [len(return_val_dict[ticker]) > 12 for index, ticker in enumerate(df_ticker["ticker"])]
])

In [18]:
sector2id = dict(zip(np.sort(list(set(sector_array))), range(len(set(sector_array)))))
industry2id = dict(zip(np.sort(list(set(industry_array))), range(len(set(industry_array)))))
use_label_data = [sector2id[sector] for sector in sector_array]
use_industry_data = [industry2id[sector] for sector in industry_array]

## Predict Using Bert

In [19]:
from NN import *
from evaluation import *

In [20]:
batch_size = 16
gradient_accumulation_steps = 1

In [21]:
len(use_label_data)

2462

In [22]:
train_data_size = 1800
valid_data_size = 262
test_data_size = (len(use_label_data) - (train_data_size + valid_data_size))

In [23]:
train_data_x = use_text_data[0:train_data_size]
train_data_y = use_label_data[0:train_data_size]
train_data_industry = use_industry_data[0:train_data_size]
train_inner_stock_mat = similarity_inner_mat[:train_data_size, :train_data_size]

In [24]:
valid_data_x = use_text_data[train_data_size:train_data_size + valid_data_size]
valid_data_y = use_label_data[train_data_size:train_data_size + valid_data_size]
valid_data_industry = use_industry_data[train_data_size:train_data_size + valid_data_size]
valid_inner_stock_mat = similarity_inner_mat[train_data_size:train_data_size + valid_data_size, 
                                                                                  train_data_size:train_data_size + valid_data_size]

In [25]:
test_data_x = use_text_data[-test_data_size:]
test_data_y = use_label_data[-test_data_size:]
test_data_industry = use_industry_data[-test_data_size:]
test_inner_stock_mat = similarity_inner_mat[-test_data_size:, -test_data_size:]

In [26]:
train_stock_label_y = np.array(use_stock_price_list)[0:train_data_size]
valid_label_y = np.array(use_stock_price_list)[train_data_size:train_data_size + valid_data_size]
test_stock_label_y = np.array(use_stock_price_list)[-test_data_size:]

In [27]:
test_stock_label_y.shape, train_stock_label_y.shape

((400,), (1800,))

In [28]:
sentence_size =    1

In [29]:
total_train_examples = len(train_data_x)

In [30]:
label_num = max(use_label_data)+ 1
industry_num = max(use_industry_data)+ 1

In [79]:
ls ../AnalysisEnglishAnnualReports/BertIndustryClustering/20200120_BERTMeanVEC/Stock_Sector_SectorName_1/

001/  01/  01_1/  1/


In [97]:
def out_actual_predicted_list(sentence_representation_all, theme_word_list, pooled_output, th_val = 0):
    actual_list = []
    predicted_list = []
    for word, theme_vector_gpu in zip(theme_word_list, pooled_output):
        theme_vector = theme_vector_gpu.detach().to("cpu").numpy()
        if np.mean(np.array(test_data_industry) == industry2id[word])  <= th_val:
            continue
        #print (word)
        doc2theme_similarity_mat = sklearn.metrics.pairwise.cosine_similarity(sentence_representation_all, theme_vector.reshape(1, -1))
        actual = np.array(range(len(test_data_industry)))[np.array(test_data_industry) == industry2id[word]]
        predicted = np.argsort(doc2theme_similarity_mat[:,0])[-1::-1]
        actual_list.append(list(actual))
        predicted_list.append(list(predicted))
    return actual_list, predicted_list 

In [246]:
def save_vectors(folda_name, use_ticker_list_train, sentence_representation_all_train, 
                use_ticker_list_valid, sentence_representation_all_valid, 
                use_ticker_list_test, sentence_representation_all_test):
    with open(folda_name + "/train_ticker_vectors.txt", "w") as f:
        for ticker, vector in zip(use_ticker_list_train, sentence_representation_all_train):
            f.write("\t".join([ticker] + [str(num) for num in vector]) + "\n")
    
    with open(folda_name + "/valid_ticker_vectors.txt", "w") as f:
        for ticker, vector in zip(use_ticker_list_valid, sentence_representation_all_valid):
            f.write("\t".join([ticker] + [str(num) for num in vector]) + "\n")
    
    with open(folda_name + "/test_ticker_vectors.txt", "w") as f:
        for ticker, vector in zip(use_ticker_list_test, sentence_representation_all_test):
            f.write("\t".join([ticker] + [str(num) for num in vector]) + "\n")

In [235]:
bert_config = BertConfig.from_pretrained('models/PreTrainigNLMbyBert/finetuned_lm/')
#model_state_dict = torch.load(read_file_name + "pytorch_model.bin")
model_state_dict = torch.load(read_file_name)
model = BertForSequenceMeanVec(bert_config, label_num, industry_num, state_dict=model_state_dict)

In [236]:
sentence_size =   1
batch_size = 16
test_data_size = len(test_data_y)

In [237]:
loss_fct = CrossEntropyLoss(ignore_index = -1)
loss_mse = MSELoss()

In [238]:
sentence_representation_all_test = evaluate_model_with_cosine_similarity(model, test_data_x, test_data_y, test_data_industry, test_data_size)

sector
same:  0.6118864
other:  0.3555874
industry
same:  0.75139105
other:  0.38757405


In [239]:
doc2soc_similarity_mat = sklearn.metrics.pairwise.cosine_similarity(sentence_representation_all_test)
sort_values = np.argsort(doc2soc_similarity_mat, axis = 1)[:,-1::-1]

## MAP@K

In [240]:
print ("sector (MAP@K)")
predicted_list = [list(item[1:]) for item in sort_values]
actual_list_all= [list(np.array(range(len(test_data_y)))[np.array(test_data_y) == test_data_y[index]]) for index in range(len(test_data_y))]
actual_list_rev = []
for index, item in enumerate(actual_list_all):
    actual_list_rev.append(list(np.array(item)[np.array(item) != index]))
for top_n in [5,10, 50]:
    print (top_n , ":",  ml_metrics.mapk(actual_list_rev, predicted_list, top_n))

print ("industry (MAP@K)")
predicted_list = [list(item[1:]) for item in sort_values]
actual_list_all= [list(np.array(range(len(test_data_industry)))[np.array(test_data_industry) == test_data_industry[index]]) for index in range(len(test_data_industry))]
actual_list_rev = []
for index, item in enumerate(actual_list_all):
    actual_list_rev.append(list(np.array(item)[np.array(item) != index]))
for top_n in [5,10, 50]:
    print (top_n , ":",  ml_metrics.mapk(actual_list_rev, predicted_list, top_n))

sector (MAP@K)
5 : 0.5308083333333333
10 : 0.4870513888888888
50 : 0.3793404444751927
industry (MAP@K)
5 : 0.31914791666666664
10 : 0.3274141424162258
50 : 0.34913729978394925


## Thene Extraction

In [241]:
theme_word_list = [word for word in industry2id]

In [242]:
theme_token_list = []
for word in theme_word_list:
    text = "[CLS] " + word +  " [SEP]"
    #text = word
    tokenized_text = tokenizer.tokenize(text)
    tokenized_id = tokenizer.convert_tokens_to_ids(tokenized_text)
    theme_token_list.append(tokenized_id)

max_seq_length_company = 8
input_ids_list = []
for tokenized_id in theme_token_list:
    input_array = np.zeros(max_seq_length_company, dtype=np.int)
    input_array[:min(max_seq_length_company, len(tokenized_id))] = tokenized_id[:min(max_seq_length_company, len(tokenized_id))]
    input_ids_list.append(input_array)

In [243]:
input_ids = torch.LongTensor(np.array(input_ids_list).astype(np.int32))
label_logits, pooled_output = model(input_ids, labels= None,  
                                                label_industry = None,
                                                labels_stock =  None, stock_vol = 0)

In [244]:
actual_list, predicted_list  = out_actual_predicted_list(sentence_representation_all_test, theme_word_list, pooled_output, 0)
print ("StockTheme")
print (len(actual_list))
for k in [5,10,50]:
        print (k, ":", ml_metrics.mapk(actual_list, predicted_list, k))

StockTheme
140
5 : 0.23416865079365076
10 : 0.25440026724975706
50 : 0.27902750448714714
